In [1]:
#create a document

from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [ ]:
#loading environment variable
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [ ]:
#creating llm mode

from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x75003b06a410>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x75003b06a290>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")     

/home/devendra/Desktop/Udemy_Generative_AI_With_Langchain_and_Huggingface_2024/Project/Langchain/langchain_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#vector store
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=documents,embedding=embeddings)
vectorstore


In [12]:
vectorstore.similarity_search("cat")

[Document(id='1f7bfc1d-47d3-49f3-924c-45a84e06e9ac', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='4ee0ae6a-a0f1-4deb-84fe-1d053411ff63', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='ee29bcc2-0118-4b26-aa99-eca4cf953f6c', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='a1e1a313-607f-47d7-a8b0-3953f5c87f84', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [13]:
# async query
await vectorstore.asimilarity_search("cat")

[Document(id='1f7bfc1d-47d3-49f3-924c-45a84e06e9ac', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='4ee0ae6a-a0f1-4deb-84fe-1d053411ff63', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='ee29bcc2-0118-4b26-aa99-eca4cf953f6c', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='a1e1a313-607f-47d7-a8b0-3953f5c87f84', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [14]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='1f7bfc1d-47d3-49f3-924c-45a84e06e9ac', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.935105562210083),
 (Document(id='4ee0ae6a-a0f1-4deb-84fe-1d053411ff63', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740900039672852),
 (Document(id='ee29bcc2-0118-4b26-aa99-eca4cf953f6c', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956902503967285),
 (Document(id='a1e1a313-607f-47d7-a8b0-3953f5c87f84', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.665792465209961)]

In [15]:
##retriever 

In [16]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda


In [17]:
#2 ways to create retriever

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["dog","cat"])

[[Document(id='4ee0ae6a-a0f1-4deb-84fe-1d053411ff63', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')],
 [Document(id='1f7bfc1d-47d3-49f3-924c-45a84e06e9ac', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]]

In [18]:
#2ndway
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs={"k":1})


retriever.batch(["cat","dog"])

[[Document(id='1f7bfc1d-47d3-49f3-924c-45a84e06e9ac', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='4ee0ae6a-a0f1-4deb-84fe-1d053411ff63', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [19]:
retriever.invoke("cat")

[Document(id='1f7bfc1d-47d3-49f3-924c-45a84e06e9ac', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [ ]:
#RAG

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer the question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human",message)])

rag_chain = {"context":retriever,"question":RunnablePassthrough()}|prompt|llm

In [21]:
response = rag_chain.invoke("tell me about the dogs")

In [22]:
response

AIMessage(content='According to the provided context, dogs are great companions, known for their loyalty and friendliness.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 85, 'total_tokens': 105, 'completion_time': 0.015000348, 'prompt_time': 0.016426255, 'queue_time': 0.409161359, 'total_time': 0.031426603}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f24ed5db-ffea-413e-9930-17c39a1fbb9c-0', usage_metadata={'input_tokens': 85, 'output_tokens': 20, 'total_tokens': 105})